Flores Lara Alberto
5BV1
Ingenieria en Inteligencia Artificial
Fecha de última modificación: 26/06/2024
Descripción detallada de la funcionalidad del programa:

El programa desarrollado tiene como objetivo realizar un análisis de sentimientos sobre un conjunto de reseñas de productos. A continuación se detalla la funcionalidad del programa, los datos requeridos y las funciones implementadas:

Datos Requeridos:

Un archivo CSV que contenga las reseñas de productos con las siguientes columnas:
ProductId: Identificador del producto.
UserId: Identificador del usuario.
Score: Puntuación dada por el usuario (1 a 5).
Summary: Resumen de la reseña.
Text: Texto completo de la reseña.
Funcionalidades Principales:

Cargar y Describir el Conjunto de Datos:
Carga el archivo CSV y muestra información general del DataFrame, incluyendo las primeras filas, dimensiones y caracterización de cada columna.

Preprocesamiento de Datos:
Filtra las columnas relevantes y convierte las puntuaciones en etiquetas de sentimiento (Negativo, Neutral, Positivo).
Balancea las clases de sentimiento para asegurar una distribución equitativa.

Limpieza de Texto:
Convierte el texto a minúsculas, elimina etiquetas HTML, direcciones de correo electrónico, URLs, signos de puntuación y números.
Elimina las stopwords y tokeniza el texto.

Stemmatización y Lematización:
Aplica técnicas de stemming y lematización para reducir las palabras a sus raíces gramaticales.

Vectorización:
Utiliza la técnica TF-IDF para convertir el texto de las reseñas en vectores numéricos.

Análisis de Sentimientos usando Diccionarios:
Implementa análisis de sentimientos utilizando los diccionarios Harvard IV-4 y Opinion Lexicon.

Análisis de Sentimientos usando Algoritmos de Aprendizaje de Máquina:
Entrena y evalúa modelos de regresión logística, árboles de decisión y máquinas de soporte vectorial (SVM) para predecir el sentimiento de las reseñas.

Análisis de Sentimientos usando Word Embeddings y Redes Neuronales:
Entrena modelos de redes neuronales convolucionales usando embeddings preentrenados (GloVe) y embeddings aprendidos a partir del conjunto de datos.

Descripción de las Funciones:

load_data(file_path): Carga el conjunto de datos desde un archivo CSV y muestra su información general.
preprocess_data(df): Filtra las columnas relevantes, convierte las puntuaciones en etiquetas de sentimiento y balancea las clases.
clean_text(text): Realiza la limpieza del texto eliminando elementos no deseados y stopwords.
stem_and_lemmatize(text): Aplica técnicas de stemming y lematización al texto.
vectorize_text(df): Convierte el texto en vectores numéricos utilizando TF-IDF.
analyze_sentiment_harvard(text): Realiza el análisis de sentimiento utilizando el diccionario Harvard IV-4.
analyze_sentiment_opinion_lexicon(text): Realiza el análisis de sentimiento utilizando el diccionario Opinion Lexicon.
train_ml_models(X_train, y_train): Entrena modelos de regresión logística, árboles de decisión y SVM.
evaluate_ml_models(X_test, y_test, models): Evalúa los modelos de aprendizaje de máquina y muestra los resultados.
train_nn_model(X_train, y_train, X_test, y_test, embedding_matrix): Entrena una red neuronal convolucional con embeddings preentrenados.
train_nn_model_learned(X_train, y_train, X_test, y_test): Entrena una red neuronal convolucional con embeddings aprendidos a partir del conjunto de datos.
cross_validate_model(model, X, y, k=5): Realiza k-fold cross-validation y retorna las precisiones del modelo.

Librerias Utilizadas:

In [1]:
# Importar las librerías necesarias
import pandas as pd
import numpy as np
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import resample
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
import pickle
import pysentiment2 as ps
from nltk.corpus import opinion_lexicon

1. Adquisición de datos
2. Análisis Exploratorio de Datos (EDA)

In [2]:
# Carga del conjunto de datos
df = pd.read_csv('Reviews.csv')

#Información general del DataFrame
print("Informacion general del DataFrame:")
print(df.info())

print("\nPrimeras filas del DataFrame:")
print(df.head())

print("\nDimensiones del DataFrame:")
print(df.shape)

print("\nCaracterizacion de cada columna:")
for column in df.columns:
    print(f"\nColumna: {column}")
    print(f"Tipo de dato: {df[column].dtype}")
    print(f"Descripcion:\n{df[column].describe()}")


Informacion general del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568454 entries, 0 to 568453
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Id                      568454 non-null  int64 
 1   ProductId               568454 non-null  object
 2   UserId                  568454 non-null  object
 3   ProfileName             568428 non-null  object
 4   HelpfulnessNumerator    568454 non-null  int64 
 5   HelpfulnessDenominator  568454 non-null  int64 
 6   Score                   568454 non-null  int64 
 7   Time                    568454 non-null  int64 
 8   Summary                 568427 non-null  object
 9   Text                    568454 non-null  object
dtypes: int64(5), object(5)
memory usage: 43.4+ MB
None

Primeras filas del DataFrame:
   Id   ProductId          UserId                      ProfileName  \
0   1  B001E4KFG0  A3SGXH7AUHU8GW                       delma

3. Preprocesamiento de Datos

In [3]:
# Justificación de las dimensiones necesarias
# Mantendremos las columnas relevantes para el análisis de sentimiento: ProductId, UserId, Score, Summary, Text
# Eliminaremos las columnas irrelevantes: Id, ProfileName, HelpfulnessNumerator, HelpfulnessDenominator, Time

columnas_a_conservar = ['Score', 'Text']
df = df[columnas_a_conservar]

# Convertir calificaciones en sentimiento
def convert_score_to_sentiment(score):
    if score in [1, 2]:
        return 'Negativo'
    elif score == 3:
        return 'Neutral'
    elif score in [4, 5]:
        return 'Positivo'

df['Sentiment'] = df['Score'].apply(convert_score_to_sentiment)

# Eliminar la columna 'Score' ya que ahora ahora es irrelevante con la nueva columna 'Sentiment'
df = df.drop(columns=['Score'])

# Observacion deel balance de las clases
print("Distribución de clases antes del balanceo:")
print(df['Sentiment'].value_counts())

# Encontramos la cantidad mínima entre las clases para balancearlas
negative = df[df['Sentiment'] == 'Negativo']
neutral = df[df['Sentiment'] == 'Neutral']
positive = df[df['Sentiment'] == 'Positivo']

min_count = min(len(negative), len(neutral), len(positive))
min_count = round(min_count/20)

negative_balanced = resample(negative, replace=False, n_samples=min_count, random_state=123)
neutral_balanced = resample(neutral, replace=False, n_samples=min_count, random_state=123)
positive_balanced = resample(positive, replace=False, n_samples=min_count, random_state=123)

df_balanced = pd.concat([negative_balanced, neutral_balanced, positive_balanced])
df= df_balanced

print("Distribución de clases después del balanceo:")
print(df_balanced['Sentiment'].value_counts())

Distribución de clases antes del balanceo:
Sentiment
Positivo    443777
Negativo     82037
Neutral      42640
Name: count, dtype: int64
Distribución de clases después del balanceo:
Sentiment
Negativo    2132
Neutral     2132
Positivo    2132
Name: count, dtype: int64


4. Limpieza de Datos

In [4]:
# Función para limpiar el texto
def clean_text(text):
    text = text.lower()  # Convertir a minúsculas
    text = re.sub(r'<.*?>', '', text)  # Eliminar etiquetas HTML
    text = re.sub(r'\S+@\S+', '', text)  # Eliminar direcciones de correo electrónico
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)  # Eliminar URLs
    text = text.translate(str.maketrans('', '', string.punctuation))  # Eliminar signos de puntuación
    text = re.sub(r'\d+', '', text)  # Eliminar números
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    filtered_text = ' '.join([word for word in word_tokens if word not in stop_words])  # Eliminar stopwords
    return filtered_text

# Aplicar la función de limpieza al texto de las reseñas
df['Cleaned_Text'] = df['Text'].apply(clean_text)

# Se muestra una comparacion entre el texto normal y despues de limpiarlo
print("Texto original:\n", df['Text'].head())
print("\nTexto limpio:\n", df['Cleaned_Text'].head())

Texto original:
 534435    Well, these capsules are sold by Top Line and ...
52570     No seriously, this is ridiculous, $45 for a ba...
92327     I like strong coffee but was able to use the c...
165543    Started out OK, but after a few weeks my dog d...
201034    The taste of raspberry is pretty much all I ge...
Name: Text, dtype: object

Texto limpio:
 534435    well capsules sold top line fulfilled amazon t...
52570     seriously ridiculous bag corn obligate carnivo...
92327     like strong coffee able use cup times reminds ...
165543    started ok weeks dog decided didnt like taste ...
201034    taste raspberry pretty much get taste smell ch...
Name: Cleaned_Text, dtype: object


5. Transformación y Normalización de Datos

In [5]:
# a. Convertir todos los términos a minúsculas (ya se hizo en la limpieza)

# b. Aplicar stemming y lematizacion al texto
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def stem_and_lemmatize(text):
    words = word_tokenize(text)
    stemmed = [stemmer.stem(word) for word in words]
    lemmatized = [lemmatizer.lemmatize(word) for word in stemmed]
    return ' '.join(lemmatized)

df['Stemmed_Lemmatized_Text'] = df['Cleaned_Text'].apply(stem_and_lemmatize)

# c. Tokenizar las reseñas
df['Tokenized_Text'] = df['Stemmed_Lemmatized_Text'].apply(word_tokenize)

# d. Vectorizar las reseñas usando TF-IDF
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Stemmed_Lemmatized_Text'])

print("Texto original:\n", df['Text'].head())
print("\nTexto limpio:\n", df['Cleaned_Text'].head())
print("\nTexto despues de aplicar stemming y lematizacion:\n", df['Stemmed_Lemmatized_Text'].head())
print("\nTexto tokenizado:\n", df['Tokenized_Text'].head())
print("\nMuestra de Matriz TF-IDF):\n", tfidf_matrix[:5])


Texto original:
 534435    Well, these capsules are sold by Top Line and ...
52570     No seriously, this is ridiculous, $45 for a ba...
92327     I like strong coffee but was able to use the c...
165543    Started out OK, but after a few weeks my dog d...
201034    The taste of raspberry is pretty much all I ge...
Name: Text, dtype: object

Texto limpio:
 534435    well capsules sold top line fulfilled amazon t...
52570     seriously ridiculous bag corn obligate carnivo...
92327     like strong coffee able use cup times reminds ...
165543    started ok weeks dog decided didnt like taste ...
201034    taste raspberry pretty much get taste smell ch...
Name: Cleaned_Text, dtype: object

Texto despues de aplicar stemming y lematizacion:
 534435    well capsul sold top line fulfil amazon top li...
52570     serious ridicul bag corn oblig carnivor brewer...
92327     like strong coffe abl use cup time remind star...
165543    start ok week dog decid didnt like tast much b...
201034    tast 

6. Análisis de Sentimientos con Diccionarios

In [6]:
# a. Análisis de sentimientos usando Harvard IV-4 con pysentiment2
harvard = ps.HIV4()

def analyze_sentiment_harvard(text):
    tokens = harvard.tokenize(text)
    score = harvard.get_score(tokens)
    return score

df['Harvard_Sentiment'] = df['Stemmed_Lemmatized_Text'].apply(lambda x: analyze_sentiment_harvard(x)['Polarity'])

# b. Análisis de sentimientos usando Opinion Lexicon con NLTK
positive_words = opinion_lexicon.positive()
negative_words = opinion_lexicon.negative()

def analyze_sentiment_opinion_lexicon(text):    
    tokens = word_tokenize(text)
    pos_score = sum(1 for word in tokens if word in positive_words)
    neg_score = sum(1 for word in tokens if word in negative_words)
    return pos_score - neg_score

df['Opinion_Lexicon_Sentiment'] = df['Stemmed_Lemmatized_Text'].apply(analyze_sentiment_opinion_lexicon)

# Mostramos algunas reseñas con sus análisis de sentimientos
print("Texto original:\n", df['Text'].head())
print("\nSentimiento Harvard IV-4:\n", df['Harvard_Sentiment'].head())
print("\nSentimiento Opinion Lexicon:\n", df['Opinion_Lexicon_Sentiment'].head())


Texto original:
 534435    Well, these capsules are sold by Top Line and ...
52570     No seriously, this is ridiculous, $45 for a ba...
92327     I like strong coffee but was able to use the c...
165543    Started out OK, but after a few weeks my dog d...
201034    The taste of raspberry is pretty much all I ge...
Name: Text, dtype: object

Sentimiento Harvard IV-4:
 534435    0.714286
52570     0.250000
92327     0.500000
165543    0.142857
201034   -0.090909
Name: Harvard_Sentiment, dtype: float64

Sentimiento Opinion Lexicon:
 534435    3
52570     1
92327     5
165543    0
201034   -4
Name: Opinion_Lexicon_Sentiment, dtype: int64


7. Análisis de Sentimientos con Algoritmos de Aprendizaje de Máquina

In [7]:
# Convertir la columna de sentimiento a un valor numérico: Negativo=0, Neutral=1, Positivo=2
sentiment_mapping = {'Negativo': 0, 'Neutral': 1, 'Positivo': 2}
df['Sentiment_Num'] = df['Sentiment'].map(sentiment_mapping)

# Vectorizamos las reseñas usando TF-IDF
tfidf_vectorizer = TfidfVectorizer()
X = tfidf_vectorizer.fit_transform(df['Stemmed_Lemmatized_Text'])
y = df['Sentiment_Num']

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# c. Regresión Logística
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train, y_train)
y_pred_logreg = logreg.predict(X_test)
print("Regresión Logística")
print("Accuracy:", accuracy_score(y_test, y_pred_logreg))
print(classification_report(y_test, y_pred_logreg))

# d. Árboles de Decisión
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)
print("Árboles de Decisión")
print("Accuracy:", accuracy_score(y_test, y_pred_dt))
print(classification_report(y_test, y_pred_dt))

# e. Máquinas de Soporte Vectorial (SVM)
svm = SVC()
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)
print("Máquinas de Soporte Vectorial")
print("Accuracy:", accuracy_score(y_test, y_pred_svm))
print(classification_report(y_test, y_pred_svm))

# Función para realizar la validación cruzada y obtener las accuracys
def cross_validate_model(model, X, y, k=5):
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    accuracies = cross_val_score(model, X, y, cv=kf, scoring='accuracy')
    return accuracies

# Realizar validación cruzada con k=5
logreg_accuracies = cross_validate_model(LogisticRegression(max_iter=1000), X, y, k=5)
dt_accuracies = cross_validate_model(DecisionTreeClassifier(), X, y, k=5)
svm_accuracies = cross_validate_model(SVC(), X, y, k=5)

print("Precisión usando Regresión Logística:", logreg_accuracies)
print("Precisión Media de Validación usando Regresión Logística:", np.mean(logreg_accuracies))

print("Precisión de Validación usando Árboles de Decisión:", dt_accuracies)
print("Precisión Media de Validación usando Árboles de Decisión:", np.mean(dt_accuracies))

print("Precisión de Validación usando SVM:", svm_accuracies)
print("Precisión Media de Validación usando SVM:", np.mean(svm_accuracies))



Regresión Logística
Accuracy: 0.66796875
              precision    recall  f1-score   support

           0       0.70      0.65      0.68       446
           1       0.56      0.63      0.59       404
           2       0.76      0.72      0.74       430

    accuracy                           0.67      1280
   macro avg       0.67      0.67      0.67      1280
weighted avg       0.67      0.67      0.67      1280

Árboles de Decisión
Accuracy: 0.51171875
              precision    recall  f1-score   support

           0       0.54      0.48      0.51       446
           1       0.45      0.50      0.47       404
           2       0.54      0.56      0.55       430

    accuracy                           0.51      1280
   macro avg       0.51      0.51      0.51      1280
weighted avg       0.51      0.51      0.51      1280

Máquinas de Soporte Vectorial
Accuracy: 0.66484375
              precision    recall  f1-score   support

           0       0.70      0.66      0.68       

8. Análisis de Sentimientos con Word Embeddings y Redes Neuronales

In [8]:
# Tokenización y Padding
tokenizer = Tokenizer(num_words=5000, oov_token='<OOV>')
tokenizer.fit_on_texts(df['Stemmed_Lemmatized_Text'])
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(df['Stemmed_Lemmatized_Text'])
padded_sequences = pad_sequences(sequences, maxlen=100, truncating='post')

# One-hot encoding de las etiquetas
sentiment_labels = pd.get_dummies(df['Sentiment_Num']).values

# Dividir el conjunto de datos en entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(padded_sequences, sentiment_labels, test_size=0.2, random_state=42)

# Cargar los embeddings preentrenados de GloVe (50 dimensiones)
embedding_index = {}
with open('glove.6B.50d.txt', encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs

embedding_dim = 50
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# Crear el modelo de red neuronal con embeddings preentrenados
model = Sequential()
model.add(Embedding(len(word_index) + 1, embedding_dim, weights=[embedding_matrix], input_length=100, trainable=False))
model.add(Conv1D(64, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Entrenar el modelo
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

# Evaluar el rendimiento del modelo
y_pred = np.argmax(model.predict(x_test), axis=-1)
y_test_labels = np.argmax(y_test, axis=-1)
print("Precisión con embeddings preentrenados:", accuracy_score(y_test_labels, y_pred))

# Crear el modelo usando una capa de embeddings aprendida a partir del cuerpo de documentos
model2 = Sequential()
model2.add(Embedding(5000, 100, input_length=100))
model2.add(Conv1D(64, 5, activation='relu'))
model2.add(GlobalMaxPooling1D())
model2.add(Dense(32, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(3, activation='softmax'))
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model2.summary()

# Entrenar el modelo
model2.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

# Evaluar el rendimiento del modelo
y_pred2 = np.argmax(model2.predict(x_test), axis=-1)
print("Accuracy with learned embeddings:", accuracy_score(y_test_labels, y_pred2))

# Crear una función para entrenar y evaluar el modelo con k-fold cross-validation
def cross_validate_model_nn(model, X, y, k=5):
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    accuracies = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)
        y_pred = np.argmax(model.predict(X_test), axis=-1)
        y_test_labels = np.argmax(y_test, axis=-1)
        accuracies.append(accuracy_score(y_test_labels, y_pred))
    return accuracies

# Evaluar ambos modelos usando k-fold cross-validation
accuracies_pretrained = cross_validate_model_nn(model, np.array(padded_sequences), np.array(sentiment_labels), k=5)
accuracies_learned = cross_validate_model_nn(model2, np.array(padded_sequences), np.array(sentiment_labels), k=5)

print("Precisión utilizando embeddings preentrenados:", accuracies_pretrained)
print("Precisión media utilizando embeddings preentrenados:", np.mean(accuracies_pretrained))
print("Precisión utilizando embeddings aprendidos:", accuracies_learned)
print("Precisión media utilizando embeddings aprendidos:", np.mean(accuracies_learned))


c:\Users\albsa\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │       865,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ ?                      │   0 (unbuilt) │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 865,400 (3.30 MB)

 Trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_1          │ ?                      │   0 (unbuilt) │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3488 - loss: 1.0951 - val_accuracy: 0.5180 - val_loss: 1.0204
Epoch 2/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5574 - loss: 0.9497 - val_accuracy: 0.6125 - val_loss: 0.8539
Epoch 3/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7328 - loss: 0.6676 - val_accuracy: 0.6453 - val_loss: 0.8197
Epoch 4/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8831 - loss: 0.3951 - val_accuracy: 0.6391 - val_loss: 0.9119
Epoch 5/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9523 - loss: 0.1932 - val_accuracy: 0.6219 - val_loss: 1.0611
Epoch 6/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9848 - loss: 0.0931 - val_accuracy: 0.6148 - val_loss: 1.2328
Epoch 7/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9901 - loss: 0.0566 - val_accuracy: 0.6125 - val_loss: 1.3946
Epoch 8/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9882 - loss: 0.0438 - val_accuracy: 0.